# Introduction

On the notebook "determining_redshifht", I used a random sample of 15000 stellar objects(5000 of stars, QSOs and galaxies) from the SDSS catalogue to determing the redshifts (z, distance measurement) of the stellar objects in the sample. In order to calculate z, I use the magnitudes of the 5 SDSS bands of each stellar object. To do this, I used two regression learning algorithms(KNearest Neighbor and Support Vector Machine). I compared the resulting values of mean squared error(mse) of the predicted values to spectroscopic z with the mse of the photometric z provided by the SDSS to its spectroscopic counterpart. I found that the mse of the last one to be 0.054 and the mse of the predicted values to be 0.186. This value which is too high, i found that it was due to the presence of higher z. Separaing the sample by z= 1, the mse of the best model for sources with z <=1 to be 0.008 and for the other subsample to be 0.099. I then used neural networks to predict the values of z which is in the notebook "determining_redshift_NN". 

In this notebook, I do the same analysis as the previous notebook but I added colour(difference between the magnitudes of two bands) information to the sample. To compare the mse of this study to the one of the initial notebook, the mse of the coloured sample is 0.149.

In [1]:
import os
import numpy as np
import pandas as pd
from pandas.plotting import table
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
sns.set()
%matplotlib inline
import tensorflow as tf
import tensorflow_docs as tfdocs
import tensorflow_docs.modeling
import tensorflow_docs.plots
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
import scikeras
from scikeras.wrappers import KerasRegressor
from keras.layers import LeakyReLU

In [2]:
#Loading files
filepath= "/home/veroastro/Documents/spectraTypes/MLphotometry/data/"
galaxy = pd.read_csv(filepath+"galaxy_nophotoZ4.csv")
star = pd.read_csv(filepath+"star_nophotoZ.csv")
QSO = pd.read_csv(filepath+"QSO_nophotoZ.csv")
#Create the data frame from the three sets of data
data = pd.concat([star, QSO, galaxy])
#Scramble the data set so that it is not ordered by spectral class (QSO, galaxy or star) 
data = data.sample(frac=1)
# Removing Id, error and class data
data_clean = data.drop(columns=['objID','err_u', 'err_g', 'err_r', 'err_i', 'err_z', 'zErr', 'class', 'subClass', 'type', 'zWarning'])
data_clean.head()

,dered_u,dered_g,dered_r,dered_i,dered_z,z
3660,19.92751,18.19411,17.42766,17.06925,16.76144,0.055536
1187,19.65844,19.45745,18.97910,18.96488,19.02022,1.275659
3414,19.43212,19.23183,19.20471,19.00803,18.99661,1.783297
678,19.54354,18.60906,18.38204,18.27748,18.27099,0.000079
4606,18.71689,18.68441,18.70579,18.49688,18.45490,1.847776


In [3]:
# Create a function to calculate the colours of the sources
def compute_colour(row):
    u = row['dered_u']
    g = row['dered_g']
    r = row['dered_r']
    i = row['dered_i']
    z = row['dered_z']
    return pd.Series({'u-g': u - g, 'g-r': g - r, 'r-i': r - i, 'i-z': i - z})
# Apply the function compute_colour
colour_ind = data_clean.apply(compute_colour, axis=1)
# Create new dataframe with the colours added
data_colour = pd.concat([data_clean,colour_ind], axis=1)
data_colour.describe()

,dered_u,dered_g,dered_r,dered_i,dered_z,z,u-g,g-r,r-i,i-z
count,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,1.500000e+04,15000.000000,15000.000000,15000.000000,15000.000000
mean,19.278773,18.242983,17.803544,17.590448,17.445029,5.338676e-01,1.035791,0.439438,0.213096,0.145420
std,1.073886,1.280207,1.444430,1.521226,1.569711,8.067329e-01,0.647352,0.350960,0.196789,0.171565
min,7.552934,7.661058,8.319589,10.746470,8.741409,5.417808e-09,-1.391450,-1.454840,-3.861471,-1.146380
25%,18.716025,17.467247,16.804580,16.486115,16.265550,3.022298e-04,0.465475,0.174497,0.087350,0.027745
50%,19.425340,18.242070,17.639700,17.365905,17.256925,8.968894e-02,1.048255,0.382455,0.202605,0.133620
75%,20.037327,19.222772,19.024073,18.929403,18.864542,9.271685e-01,1.506298,0.715958,0.372275,0.273913
max,21.546420,22.145940,21.324990,21.340340,20.616610,6.164721e+00,3.653620,3.852890,1.688770,3.762850


In [4]:
# creating the features and label dataframes and normalising X
from sklearn.preprocessing import RobustScaler
X = data_colour.drop(['z'], axis=1)
y = data_colour['z']
scaler = RobustScaler()
X_t = scaler.fit_transform(X)

In [5]:
# divide the X and y dataframsn into train and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_t, y, 
                                                    test_size=0.3, random_state=42)

In [6]:
parameters=[
    {
        "name": "learning_rate",
        "type": "range",
        "bounds": [0.001, 0.5],
        "log_scale": True,
    },
    {
        "name": "dropout_rate",
        "type": "range",
        "bounds": [0.01, 0.9],
        "log_scale": True,
    },
    {
        "name": "num_hidden_layers",
        "type": "range",
        "bounds": [1, 7],
        "value_type": "int"
    },
    {
        "name": "neurons_per_layer",
        "type": "range",
        "bounds": [1, 300],
        "value_type": "int"
    },
    {
        "name": "batch_size",
        "type": "choice",
        "values": [8, 10, 16, 20, 30],
    },
    
    {
        "name": "activation",
        "type": "choice",
        "values": [ 'LeakyReLU(alpha=0.3)', 'relu'],
    },
    {
        "name": "optimizer",
        "type": "choice",
        "values": ['adam', 'rms', 'sgd'],
    },
]

In [7]:
# This returns a multi-layer-perceptron model in Keras.
def get_keras_model(num_hidden_layers, 
                    num_neurons_per_layer, 
                    dropout_rate, 
                    activation):
    # create the MLP model.
    
    # define the layers.
    inputs = tf.keras.Input(shape=(X_train.shape[1],))  # input layer.
    x = Dropout(dropout_rate)(inputs) # dropout on the weights.
    
    # Add the hidden layers.
    for i in range(num_hidden_layers):
        x = Dense(num_neurons_per_layer, 
                         activation=activation)(x)
        x = Dropout(dropout_rate)(x)
    
    # output layer.
    outputs = Dense(1, activation='linear')(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

In [8]:
# This function takes in the hyperparameters and returns a score (Cross validation). 
# Returns the mean of the validation loss based on which we decide which algorithm has the best hyperparameters
def keras_mlp_cv_score(parameterization, weight=None):
    
    model = get_keras_model(parameterization.get('num_hidden_layers'),
                            parameterization.get('neurons_per_layer'),
                            parameterization.get('dropout_rate'),
                            parameterization.get('activation'))
    
    opt = parameterization.get('optimizer')
    opt = opt.lower()
    
    learning_rate = parameterization.get('learning_rate')
    
    if opt == 'adam':
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif opt == 'rms':
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    else:
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
        
    act = parameterization.get('activation')
    act = act.lower()
    
    if act == 'leakyrelu': 
        activation = ""
        get_keras_model.add(leakyReLU())
    
    
    NUM_EPOCHS = 100
    
    # Specify the training configuration.
    model.compile(optimizer=optimizer,
                  loss=tf.keras.losses.MeanSquaredError(),
                  metrics=['mae', 'mse'] )

    data = X_train
    labels = y_train.values
    
    early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

    
    # fit the model using a 20% validation set. with a patience of 10 to avoid overfitting
    res = model.fit(data, labels, epochs=NUM_EPOCHS, batch_size=parameterization.get('batch_size'),
                    validation_split=0.2, callbacks=[early_stop, tfdocs.modeling.EpochDots()])
    
    # look at the last 10 epochs. Get the mean and standard deviation of the validation score.
    last10_scores = np.array(res.history['val_loss'][-10:])
    mean = last10_scores.mean()
    sem = last10_scores.std()
    
    # If the model didn't converge then set a high loss.
    if np.isnan(mean):
        return 9999.0, 0.0
    
    return mean, sem

In [9]:
import optuna

# Define the objective function for Optuna.
def objective(trial):
    # Sample hyperparameters from the ranges defined in `parameters`.
    learning_rate = trial.suggest_loguniform('learning_rate', 0.001, 0.5)
    dropout_rate = trial.suggest_loguniform('dropout_rate', 0.01, 0.9)
    num_hidden_layers = trial.suggest_int('num_hidden_layers', 1, 7)
    neurons_per_layer = trial.suggest_int('neurons_per_layer', 1, 300)
    batch_size = trial.suggest_categorical('batch_size', [8, 10, 16, 20, 30])
    activation = trial.suggest_categorical('activation', [ 'relu'])
    optimizer = trial.suggest_categorical('optimizer', ['adam', 'rms', 'sgd'])

    # Set the hyperparameters in a dictionary to pass to `keras_mlp_cv_score`.
    parameterization = {
        'learning_rate': learning_rate,
        'dropout_rate': dropout_rate,
        'num_hidden_layers': num_hidden_layers,
        'neurons_per_layer': neurons_per_layer,
        'batch_size': batch_size,
        'activation': activation,
        'optimizer': optimizer
    }

    # Call `keras_mlp_cv_score` to get the cross-validation score.
    score, _ = keras_mlp_cv_score(parameterization)

    return score

# Use Optuna to optimize the neural network.
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

# Get the best hyperparameters and the best score.
best_params = study.best_params
best_score = study.best_value


[I 2023-04-27 08:43:12,307] A new study created in memory with name: no-name-59684a1a-9e9e-4886-9486-9bbc4189a09e
/tmp/ipykernel_134456/649837142.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 0.001, 0.5)
/tmp/ipykernel_134456/649837142.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  dropout_rate = trial.suggest_loguniform('dropout_rate', 0.01, 0.9)
2023-04-27 08:43:17.940713: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-04-27 08:43:17.940745: I tensorflow/stream_executor/cuda/cuda_diag

Epoch 1/100
839/840 [============================>.] - ETA: 0s - loss: 3.1595 - mae: 0.6588 - mse: 3.1595
Epoch: 0, loss:3.1574,  mae:0.6592,  mse:3.1574,  val_loss:0.6474,  val_mae:0.6381,  val_mse:0.6474,  
840/840 [==============================] - 3s 3ms/step - loss: 3.1574 - mae: 0.6592 - mse: 3.1574 - val_loss: 0.6474 - val_mae: 0.6381 - val_mse: 0.6474
Epoch 2/100
840/840 [==============================] - 2s 2ms/step - loss: 1.0388 - mae: 0.6709 - mse: 1.0388 - val_loss: 0.6456 - val_mae: 0.6635 - val_mse: 0.6456
Epoch 3/100
840/840 [==============================] - 2s 3ms/step - loss: 0.6626 - mae: 0.6638 - mse: 0.6626 - val_loss: 0.6527 - val_mae: 0.6219 - val_mse: 0.6527
Epoch 4/100
840/840 [==============================] - 2s 3ms/step - loss: 0.6571 - mae: 0.6619 - mse: 0.6571 - val_loss: 0.6492 - val_mae: 0.6312 - val_mse: 0.6492
Epoch 5/100
840/840 [==============================] - 2s 2ms/step - loss: 0.6546 - mae: 0.6613 - mse: 0.6546 - val_loss: 0.6741 - val_mae: 0.7

[I 2023-04-27 08:44:13,521] Trial 0 finished with value: 0.6521165072917938 and parameters: {'learning_rate': 0.07935912349515462, 'dropout_rate': 0.014928486176737114, 'num_hidden_layers': 6, 'neurons_per_layer': 51, 'batch_size': 10, 'activation': 'relu', 'optimizer': 'adam'}. Best is trial 0 with value: 0.6521165072917938.


Epoch 1/100
824/840 [============================>.] - ETA: 0s - loss: 0.3940 - mae: 0.4034 - mse: 0.3940
Epoch: 0, loss:0.3925,  mae:0.4032,  mse:0.3925,  val_loss:0.2973,  val_mae:0.3577,  val_mse:0.2973,  
840/840 [==============================] - 3s 3ms/step - loss: 0.3925 - mae: 0.4032 - mse: 0.3925 - val_loss: 0.2973 - val_mae: 0.3577 - val_mse: 0.2973
Epoch 2/100
840/840 [==============================] - 1s 2ms/step - loss: 0.3160 - mae: 0.3655 - mse: 0.3160 - val_loss: 0.2748 - val_mae: 0.3373 - val_mse: 0.2748
Epoch 3/100
840/840 [==============================] - 2s 2ms/step - loss: 0.3001 - mae: 0.3509 - mse: 0.3001 - val_loss: 0.2615 - val_mae: 0.3217 - val_mse: 0.2615
Epoch 4/100
840/840 [==============================] - 3s 3ms/step - loss: 0.2879 - mae: 0.3372 - mse: 0.2879 - val_loss: 0.2529 - val_mae: 0.3086 - val_mse: 0.2529
Epoch 5/100
840/840 [==============================] - 2s 2ms/step - loss: 0.2875 - mae: 0.3312 - mse: 0.2875 - val_loss: 0.2474 - val_mae: 0.3

840/840 [==============================] - 1s 2ms/step - loss: 0.2091 - mae: 0.2533 - mse: 0.2091 - val_loss: 0.1615 - val_mae: 0.2093 - val_mse: 0.1615
Epoch 99/100
840/840 [==============================] - 1s 2ms/step - loss: 0.2084 - mae: 0.2511 - mse: 0.2084 - val_loss: 0.1595 - val_mae: 0.2084 - val_mse: 0.1595
Epoch 100/100
840/840 [==============================] - 1s 2ms/step - loss: 0.2075 - mae: 0.2514 - mse: 0.2075 - val_loss: 0.1592 - val_mae: 0.2084 - val_mse: 0.1592


...

[I 2023-04-27 12:02:28,055] Trial 97 finished with value: 0.12167855203151703 and parameters: {'learning_rate': 0.001304820833101806, 'dropout_rate': 0.031514938262801315, 'num_hidden_layers': 6, 'neurons_per_layer': 234, 'batch_size': 8, 'activation': 'relu', 'optimizer': 'sgd'}. Best is trial 59 with value: 0.11328117623925209.


Epoch 1/100
831/840 [============================>.] - ETA: 0s - loss: 0.4394 - mae: 0.4631 - mse: 0.4394
Epoch: 0, loss:0.4380,  mae:0.4619,  mse:0.4380,  val_loss:0.2752,  val_mae:0.3337,  val_mse:0.2752,  
840/840 [==============================] - 3s 3ms/step - loss: 0.4380 - mae: 0.4619 - mse: 0.4380 - val_loss: 0.2752 - val_mae: 0.3337 - val_mse: 0.2752
Epoch 2/100
840/840 [==============================] - 2s 3ms/step - loss: 0.2832 - mae: 0.3203 - mse: 0.2832 - val_loss: 0.2486 - val_mae: 0.2949 - val_mse: 0.2486
Epoch 3/100
840/840 [==============================] - 2s 3ms/step - loss: 0.2688 - mae: 0.3021 - mse: 0.2688 - val_loss: 0.2356 - val_mae: 0.2796 - val_mse: 0.2356
Epoch 4/100
840/840 [==============================] - 2s 3ms/step - loss: 0.2564 - mae: 0.2895 - mse: 0.2564 - val_loss: 0.2241 - val_mae: 0.2713 - val_mse: 0.2241
Epoch 5/100
840/840 [==============================] - 2s 3ms/step - loss: 0.2468 - mae: 0.2805 - mse: 0.2468 - val_loss: 0.2131 - val_mae: 0.2

[I 2023-04-27 12:05:28,423] Trial 98 finished with value: 0.11878685355186462 and parameters: {'learning_rate': 0.0017131707829217756, 'dropout_rate': 0.01851876225337959, 'num_hidden_layers': 6, 'neurons_per_layer': 209, 'batch_size': 10, 'activation': 'relu', 'optimizer': 'sgd'}. Best is trial 59 with value: 0.11328117623925209.


Epoch 1/100
1046/1050 [============================>.] - ETA: 0s - loss: 0.3159 - mae: 0.3458 - mse: 0.3159
Epoch: 0, loss:0.3154,  mae:0.3455,  mse:0.3154,  val_loss:0.2294,  val_mae:0.2725,  val_mse:0.2294,  
1050/1050 [==============================] - 4s 3ms/step - loss: 0.3154 - mae: 0.3455 - mse: 0.3154 - val_loss: 0.2294 - val_mae: 0.2725 - val_mse: 0.2294
Epoch 2/100
1050/1050 [==============================] - 3s 3ms/step - loss: 0.2554 - mae: 0.2883 - mse: 0.2554 - val_loss: 0.2153 - val_mae: 0.2598 - val_mse: 0.2153
Epoch 3/100
1050/1050 [==============================] - 3s 3ms/step - loss: 0.2395 - mae: 0.2705 - mse: 0.2395 - val_loss: 0.1892 - val_mae: 0.2443 - val_mse: 0.1892
Epoch 4/100
1050/1050 [==============================] - 3s 3ms/step - loss: 0.2279 - mae: 0.2613 - mse: 0.2279 - val_loss: 0.1750 - val_mae: 0.2251 - val_mse: 0.1750
Epoch 5/100
1050/1050 [==============================] - 3s 3ms/step - loss: 0.2148 - mae: 0.2504 - mse: 0.2148 - val_loss: 0.1684 - 

[I 2023-04-27 12:08:14,727] Trial 99 finished with value: 0.12465600222349167 and parameters: {'learning_rate': 0.005019962381559987, 'dropout_rate': 0.026068624202812255, 'num_hidden_layers': 5, 'neurons_per_layer': 272, 'batch_size': 8, 'activation': 'relu', 'optimizer': 'sgd'}. Best is trial 59 with value: 0.11328117623925209.


In [10]:
print(best_params, best_score)

{'learning_rate': 0.003931416839453604, 'dropout_rate': 0.024815972888615808, 'num_hidden_layers': 6, 'neurons_per_layer': 217, 'batch_size': 16, 'activation': 'relu', 'optimizer': 'sgd'} 0.11328117623925209


In [11]:

# Build the Keras model with the best hyperparameters
model = get_keras_model(best_params['num_hidden_layers'], 
                        best_params['neurons_per_layer'], 
                        best_params['dropout_rate'], 
                        best_params['activation'])

opt = best_params['optimizer']
opt = opt.lower()

learning_rate = best_params['learning_rate']

if opt == 'adam':
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
elif opt == 'rms':
    optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
else:
    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)

model.compile(optimizer=optimizer,
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['mae', 'mse'])

# Train the model on the entire training set
nn= model.fit(X_train, y_train, epochs=100, batch_size=best_params['batch_size'])


Epoch 1/100
657/657 [==============================] - 3s 3ms/step - loss: 0.3703 - mae: 0.3948 - mse: 0.3703
Epoch 2/100
657/657 [==============================] - 2s 3ms/step - loss: 0.2563 - mae: 0.2956 - mse: 0.2563
Epoch 3/100
657/657 [==============================] - 2s 3ms/step - loss: 0.2404 - mae: 0.2780 - mse: 0.2404
Epoch 4/100
657/657 [==============================] - 2s 3ms/step - loss: 0.2274 - mae: 0.2647 - mse: 0.2274
Epoch 5/100
657/657 [==============================] - 2s 3ms/step - loss: 0.2184 - mae: 0.2564 - mse: 0.2184
Epoch 6/100
657/657 [==============================] - 2s 3ms/step - loss: 0.2143 - mae: 0.2505 - mse: 0.2143
Epoch 7/100
657/657 [==============================] - 2s 3ms/step - loss: 0.2076 - mae: 0.2441 - mse: 0.2076
Epoch 8/100
657/657 [==============================] - 2s 3ms/step - loss: 0.1996 - mae: 0.2364 - mse: 0.1996
Epoch 9/100
657/657 [==============================] - 2s 3ms/step - loss: 0.2009 - mae: 0.2352 - mse: 0.2009
Epoch 10/1

In [12]:
from sklearn.metrics import mean_squared_error
y_pred = model.predict(X_test)

print('Mean Square Error:', round(mean_squared_error(y_test,y_pred), 3))

Mean Square Error: 0.132


# Conclusions

The mse of 0.132 is the best value of mse obtained so far (KNN 0.186, KNN colour 0.149, NN 0.145). But it is still to high with respect of the SDSS photometric redshifht of 0.054. This could be due to the fact that there are not enough sources with z > 1.